<h1> Set-up </h1>

In [1]:
import os
import sys
import json
import openai
from langchain.tools import tool
from pydantic.v1 import BaseModel, Field
from IPython.display import display, HTML
from langchain.chat_models import ChatOpenAI
from langchain.schema.agent import AgentFinish
from langchain.prompts import ChatPromptTemplate
from langchain.prompts import MessagesPlaceholder
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_core.tools import Tool
sys.path.append('../..')

In [2]:
# Setting your personal OPENAI Api Key
openai.api_key = os.environ['OPENAI_API_KEY']

<h1> Initializing Tools </h1>

In [3]:
class Design_logo_input(BaseModel):
    topic: str = Field(..., description="The topic for which the logo should be designed for. For example, if the topic were 'ice-cream shop' then the logo returned would be a logo fit for the ice-cream shop to put on their sign and merchandise.")

@tool(args_schema=Design_logo_input)
def design_logo_concepts(topic: str) -> str:
    """Returns the url for a logo design that is based on the provided topic"""
    # Currently a very empty function
    return f"s3://designs/logo_{len(topic)%5+1}.jpg"

In [4]:
class UI_Output_Request(BaseModel):
    inpt: str = Field(... , description=f"A description of the UI function you want and how you would call it along with the arguments you would use")

@tool(args_schema=UI_Output_Request)
def ui_output_request(inpt: str) -> str:
    """Provides a convenient way for you to describe what UI function you want and what to call it with without actually calling a UI function."""
    return inpt

In [5]:
class Produce_UI_input(BaseModel):
    path: str = Field(... , description=f"The path of the image for the UI page. If no image found provide an empty string.")
    link: str = Field(... , description=f"The link to the webpage where the image is found.")
#@tool(args_schema=Product_UI_input)
#def produce_image_UI(path: str) -> str:
#    """This is a UI function. Produces the JSON for a general UI page about an image."""
#    data = {"component":"Image", "props":{"url":path}}
#    return  json.dumps(data)

@tool(args_schema=Product_UI_input)
def produce_block_ui(image_path: str, link: str) -> str
    

In [ ]:
search = GoogleSearchAPIWrapper()

tool = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

<h1> Registering Tools </h1>

In [6]:
tools = [design_logo_concepts]
functions = [convert_to_openai_function(f) for f in tools]

In [7]:
ui_tools = [produce_image_UI, search]
ui_functions = [convert_to_openai_function(f) for f in ui_tools]

In [8]:
tool_dictionary = {
        "design_logo_concepts": design_logo_concepts
    }

In [9]:
ui_tool_dictionary = {
    "produce_image_UI": produce_image_UI
}

<h1> Setting up LLM Prompts </h1>

In [10]:
agent_system_message = "You are helpful assistant."

In [11]:
cognitivie_ui_instruction_I = """Based on the Chat History, think step by step. 
    What does the user want based on Chat history?
    Answer (No) if you think the responses should not be an HTML page.
    Now, if you think the responses should be an HTML page, what should be on the HTML page?
    Do you have enough information already based on the chat history?
    Answer (Yes) if you think Chat History has every information you need to generate the HTML page. 
    Answer (More) if you think you need more information from the user.
    Answer should only be No, Yes or More."""

In [12]:
cognitive_ui_instruction_UI = "Your task is to choose a function from the available options that best matches your ideal UI page" +\
    " that will serve as output for information from the scratch pad. Return a function call to it with the appropriate arguments" +\
    " and nothing else."

In [13]:
cognitive_ui_instruction_html = """Your task is to generate HTML code based on instructions from the (HumanMessage) in chat history. 
    Produce fully functioning and standalone HTML code using information in the agent_scratchpad and chat history (only). 
    (No) make up information that is not in the agent_scratchpad and chat history. 
    ((Output only the HTML code and nothing else)). Output Your HTML starting now:"""


In [14]:
cognitive_ui_instruction_more_info = """Based on chat history, think step by step.
    What should be on the HTML page based on the user's need?
    Do you already have the information?
    if not, ask user one question so that you can get the information you want to put on the HTML page.
    Output only the question you want to ask."""

In [15]:
cognitive_ui_instruction_text = "Answer what is in the scratchpad. Do not comment on this instruction."

<h1> Initializing Chains + Agent </h1>

In [16]:
gpt_version = "gpt-3.5-turbo"

In [17]:
init_prompt_model = ChatOpenAI(model_name= gpt_version, temperature=0).bind(functions=functions)
cognitive_ui_model = ChatOpenAI(model_name=gpt_version, temperature=0).bind(functions=ui_functions)
no_func_model = ChatOpenAI(model_name=gpt_version, temperature=0)

/opt/conda/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
MEMORY_KEY = "chat_history"
from langchain_core.messages import AIMessage, HumanMessage
chat_history = []

prompt = ChatPromptTemplate.from_messages([
    ("system", agent_system_message),
    MessagesPlaceholder(variable_name=MEMORY_KEY),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [19]:
init_prompt_chain = prompt | init_prompt_model | OpenAIFunctionsAgentOutputParser()
cognitivie_ui_chain =  prompt | cognitive_ui_model | OpenAIFunctionsAgentOutputParser()
no_func_chain = prompt | no_func_model | OpenAIFunctionsAgentOutputParser()

<h1> UI Output Generators </h1>

In [20]:
def produce_ui_output(user_input, intermediate_steps):
    result = cognitivie_ui_chain.invoke({
        "input": cognitive_ui_instruction_UI,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps),
        "chat_history": chat_history,
    })
    
    tool = ui_tool_dictionary[result.tool]
    observation = tool.run(result.tool_input)
    return observation

In [21]:
def produce_html(user_input, intermediate_steps):
    result = no_func_chain.invoke({
        "input": cognitive_ui_instruction_html,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps),
        "chat_history": chat_history,
    })
    
    return result

<h1> Agent Runnable </h1>

In [22]:
def run_agent(user_input, ui_producer=produce_ui_output):
    """
    Runs the UI agent which will return a tuple of (LLM Response, Is it calling a UI Function)
    """
    intermediate_steps = []
    #chat_history=[]
    result = init_prompt_chain.invoke({
        "input": user_input,
        "agent_scratchpad": format_to_openai_functions(intermediate_steps),
        "chat_history": chat_history,
    })
    chat_history.extend(
        [
            HumanMessage(content=user_input),
        ]
    )
    # Are we calling the functions?
    if not isinstance(result, AgentFinish):
        # Run first non-UI function
        tool = tool_dictionary[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))
    # Cognitive UI Steps
    result = no_func_chain.invoke({
        "input": cognitivie_ui_instruction_I + user_input,
        "agent_scratchpad": [],
        "chat_history": chat_history,
    })
    print(result)
    if result.log.lower() == "yes":
        return ui_producer(user_input, intermediate_steps), True
    elif result.log.lower() == "more":
        result = no_func_chain.invoke({
            "input": cognitive_ui_instruction_more_info,
            "agent_scratchpad": [],
            "chat_history": chat_history,
        })
        return result, True
    else:
        result = no_func_chain.invoke({
            "input": cognitive_ui_instruction_text,
            "agent_scratchpad": [],
            "chat_history": chat_history,
        })
        return result, False
        

<h1>TEST HTML</h1>

In [23]:
#html_code = run_agent("Design me a big red logo", produce_html)[0].return_values["output"]
#display(HTML(html_code))
re = run_agent("Design me a big red logo", produce_html)[0].log
re

return_values={'output': 'More'} log='More'


'What text or symbol would you like to be included in the big red logo design?'

In [24]:
chat_history

[HumanMessage(content='Design me a big red logo')]

In [25]:
re = run_agent("I need a red bear.", produce_html)[0].log
re

return_values={'output': 'More'} log='More'


'What text or slogan would you like to include in the logo design with the red bear?'

In [40]:
re = run_agent("No text on the logo.", produce_html)[0].log
re

return_values={'output': 'Yes'} log='Yes'


'```html\n<!DOCTYPE html>\n<html lang="en">\n<head>\n<meta charset="UTF-8">\n<meta name="viewport" content="width=device-width, initial-scale=1.0">\n<title>Red Bear Logo</title>\n<style>\n    body {\n        background-color: white;\n        text-align: center;\n    }\n    img {\n        max-width: 100%;\n        height: auto;\n    }\n</style>\n</head>\n<body>\n    <img src="s3://designs/logo_4.jpg" alt="Red Bear Logo">\n</body>\n</html>\n```'

In [35]:
chat_history

[HumanMessage(content='Design me a big red logo'),
 HumanMessage(content='I need a red bear.'),
 HumanMessage(content='No text.')]

In [27]:
#display(HTML(re))

In [41]:
display(HTML(''))

In [26]:
chat_history=[]

In [27]:
user_input = """I want a Resume."""
re = run_agent(user_input, produce_html)[0].log
re
#display(HTML(html_code))

return_values={'output': 'More'} log='More'


'What is your full name?'

In [28]:
chat_history

[HumanMessage(content='I want a Resume.')]

In [29]:
user_input = """I am Yeo Zheng."""
re = run_agent(user_input, produce_html)[0].log
re

return_values={'output': 'More'} log='More'


'What is your professional background and work experience that you would like to include in your resume?'

In [30]:
user_input = """I am an MEng Student at Cornell University from 2023 to 2024. I studied Computer Science as my major. I also worked
                at GEN AI lab from 2023 to 2024. It was a great experience."""
re = run_agent(user_input, produce_html)[0].log
re

return_values={'output': 'Yes'} log='Yes'


KeyboardInterrupt: 

In [ ]:
display(HTML(re))

In [ ]:
user_input = """My position at GenAI lab is a ML ops Enginner."""
re = run_agent(user_input, produce_html)[0].log
re

In [ ]:
user_input = """I don't want anything else. This should be enough already."""
re = run_agent(user_input, produce_html)[0].log
re

In [74]:
display(HTML(re))

<h1>Conclusions</h1>